# Deep Convolutional Neural Netword AlexNet

## AlexNet Model

In [54]:
import d2lzh as d2l
from mxnet import nd, init, gluon
from mxnet.gluon import nn, data as gdata
import os
import sys
import mxnet as mx

In [94]:
net = nn.Sequential()

net.add(nn.Conv2D(24, kernel_size=5, strides=2, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2),
       nn.Conv2D(64, kernel_size=5, padding=2, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2),
       nn.Conv2D(96, kernel_size=2, padding=1, activation='relu'),
       nn.Conv2D(96, kernel_size=3, padding=1, activation='relu'),
       nn.Conv2D(96, kernel_size=3, padding=1, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2),
       nn.Dense(192, activation='relu'), nn.Dropout(0.5),
       nn.Dense(192, activation='relu'), nn.Dropout(0.5),
       nn.Dense(10))

In [95]:
X = nd.random.uniform(shape=(1, 1, 28, 28))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv50 output shape:	 (1, 24, 12, 12)
pool30 output shape:	 (1, 24, 5, 5)
conv51 output shape:	 (1, 64, 5, 5)
pool31 output shape:	 (1, 64, 2, 2)
conv52 output shape:	 (1, 96, 3, 3)
conv53 output shape:	 (1, 96, 3, 3)
conv54 output shape:	 (1, 96, 3, 3)
pool32 output shape:	 (1, 96, 1, 1)
dense30 output shape:	 (1, 192)
dropout20 output shape:	 (1, 192)
dense31 output shape:	 (1, 192)
dropout21 output shape:	 (1, 192)
dense32 output shape:	 (1, 10)


## Read Dataset

In [13]:
def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'fashion-mnist')):
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)
    mnisi_train = gdata.vision.FashionMNIST(root=root, train=True)
    mnisi_test = gdata.vision.FashionMNIST(root=root, train=False)
    num_worker = 0 if sys.platform.startswith('win') else 4
    train_iter = gdata.DataLoader(
        mnisi_train.transform_first(transformer), batch_size, shuffle=True,
        num_workers=num_worker
    )
    test_iter = gdata.DataLoader(
        mnisi_test.transform_first(transformer), batch_size, shuffle=True,
        num_workers=num_worker
    )
    return train_iter, test_iter

In [96]:
batch_size = 1
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=None)

## Train Model

In [97]:
lr, num_epochs, ctx = 0.001, 5, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=mx.gpu(), init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr})
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, mx.gpu(), num_epochs)

training on gpu(0)


MXNetError: [15:31:54] c:\jenkins\workspace\mxnet-tag\mxnet\src\storage\./pooled_storage_manager.h:157: cudaMalloc failed: out of memory